In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import GetData as data

2024-05-15 15:34:27.696156: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-15 15:34:27.729563: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 15:34:27.729581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 15:34:27.730671: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 15:34:27.737270: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_user1_220617_bag, labels_user1_220617_bag = data.get_data_from_datasets('/home/cmonedero/TFG/User1/Motion/User1_220617_Bag.csv', '/home/cmonedero/TFG/User1/220617/Label_New_User1_220617.txt')

Hecho!



In [4]:
x_windows_220617_bag, y_windows_220617_bag = data.divide_data_arrays_into_windows(data_user1_220617_bag, labels_user1_220617_bag, moda=True, window_size=500)
print("Tamaño ventanas X --> ", x_windows_220617_bag.shape)
print("Tamaño ventanas Y --> ", y_windows_220617_bag.shape)

Hecho!

Tamaño ventanas X -->  (3651, 500, 6)
Tamaño ventanas Y -->  (3651,)


In [5]:
x_train_220617_bag, y_train_220617_bag, x_test_220617_bag, y_test_220617_bag = data.divide_train_test(x_windows_220617_bag, y_windows_220617_bag, 0.7)
print(f'Shape x_train --> {x_train_220617_bag.shape}')
print(f'Shape y_train --> {y_train_220617_bag.shape}')
print(f'Shape x_test --> {x_test_220617_bag.shape}')
print(f'Shape y_train --> {y_test_220617_bag.shape}')

Shape x_train --> (2555, 500, 6)
Shape y_train --> (2555,)
Shape x_test --> (1096, 500, 6)
Shape y_train --> (1096,)


In [6]:
print(np.unique(y_train_220617_bag))

[1 2 5 6]


In [7]:
inputs = Input(shape=(500,6))
x = inputs
x = Flatten()(x)
print(f'Shape del input despues de Flatten {x.shape}')
x = Dense(500*6, activation='relu')(x)
print(f'Shape de la primera capa {x.shape}')
x = Dense(1000, activation='relu')(x)
print(f'Shape de la capa intermedia {x.shape}')
x = Dense(500, activation='relu')(x)
print(f'Shape de la segunda capa intermedia {x.shape}')
x = Dense(500, activation='relu')(x)
print(f'Shape de la tercera capa intermedia {x.shape}')
outputs = Dense(8, activation='softmax')(x)
print(f'Shape de la capa final {outputs.shape}')

modelo = keras.Model(inputs, outputs)

modelo.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

Shape del input despues de Flatten (None, 3000)
Shape de la primera capa (None, 3000)
Shape de la capa intermedia (None, 1000)
Shape de la segunda capa intermedia (None, 500)
Shape de la tercera capa intermedia (None, 500)
Shape de la capa final (None, 8)


2024-05-15 15:35:01.511676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 970 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6
2024-05-15 15:35:01.514179: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 40933 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:98:00.0, compute capability: 8.6


In [10]:
modelo.fit(tf.constant(x_train_220617_bag), tf.cast(tf.constant(y_train_220617_bag), tf.float32), epochs=10)

Epoch 1/10
19/80 [======>.......................] - ETA: 0s - loss: 0.1076 - accuracy: 0.9589

80/80 [==============================] - 1s 8ms/step - loss: 0.0786 - accuracy: 0.9726
Epoch 2/10
80/80 [==============================] - 1s 9ms/step - loss: 0.0565 - accuracy: 0.9773
Epoch 3/10
80/80 [==============================] - 1s 9ms/step - loss: 0.0484 - accuracy: 0.9840
Epoch 4/10
80/80 [==============================] - 1s 9ms/step - loss: 0.0399 - accuracy: 0.9859
Epoch 5/10
80/80 [==============================] - 1s 9ms/step - loss: 0.0387 - accuracy: 0.9863
Epoch 6/10
80/80 [==============================] - 1s 9ms/step - loss: 0.1215 - accuracy: 0.9750
Epoch 7/10
80/80 [==============================] - 1s 9ms/step - loss: 0.2941 - accuracy: 0.9503
Epoch 8/10
80/80 [==============================] - 1s 9ms/step - loss: 0.1842 - accuracy: 0.9636
Epoch 9/10
80/80 [==============================] - 1s 9ms/step - loss: 0.5084 - accuracy: 0.9577
Epoch 10/10
80/80 [==============================] - 1s 9ms/step - loss: 0.0811 - accuracy: 0.9746


In [11]:
modelo.evaluate(x=x_test_220617_bag, y=y_test_220617_bag)

35/35 [==============================] - 0s 5ms/step - loss: 152.5936 - accuracy: 0.5712


[152.59364318847656, 0.5711678862571716]

In [12]:
modelo.save('PerceptronMemo/perceptron.h5')

/home/cmonedero/TFG/tfg_env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
